In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
%%time
input_data=[]
output_data=[]
test_data=[]
x_test_label=[]
for dirpath, dirnames, filenames in os.walk('./stage1_train'):
    if dirpath[-6:]=='images':
        #print(dirpath)
        input_data.append(dirpath+"/"+filenames[0])
    if dirpath[-5:]=='masks':
        for item in filenames:
            filenames[filenames.index(item)]=dirpath+"/"+item
        output_data.append(filenames)
#len(dirpath)
for dirpath, dirnames, filenames in os.walk('./stage1_test'):
    if dirpath[-6:]=='images':
        test_data.append(dirpath+"/"+filenames[0])
        x_test_label.append(filenames[0][:-4])

CPU times: user 95 ms, sys: 124 ms, total: 219 ms
Wall time: 428 ms


In [3]:
%%time
x_train=[]
for path in input_data:
    img=mpimg.imread(path)
    x_train.append(img)
    #plt.imshow(img[:,:,1])

y_train=[]
for ans in output_data:
    tmp=0
    for i in (ans):
        img=mpimg.imread(i)
        tmp+=img
    #plt.imshow(tmp)
    y_train.append(tmp)
    
x_test=[]
x_test_label=[]
for path in test_data:
    img=mpimg.imread(path)
    x_test.append(img)
    #plt.imshow(img[:,:,1])

CPU times: user 1min 56s, sys: 5.39 s, total: 2min 1s
Wall time: 2min 7s


## CNN model

In [4]:
import keras as K
from keras.layers import Input, Conv2D, MaxPool2D, Deconv2D, UpSampling2D
from keras.models import Model
input_img=Input(shape=(None,None,4))
conv_1=Conv2D(8,kernel_size=(2,2),activation='relu')
maxpool_1=MaxPool2D((2,2))
conv_2=Conv2D(16,kernel_size=(2,2),activation='relu')
maxpool_2=MaxPool2D((2,2))
conv_3=Conv2D(32,kernel_size=(2,2),activation='relu')
maxpool_3=MaxPool2D((2,2))

deconv_1=Deconv2D(32,kernel_size=(2,2),activation='relu')
upsamp_1=UpSampling2D((2,2))
deconv_2=Deconv2D(16,kernel_size=(2,2),activation='relu')
upsamp_2=UpSampling2D((2,2))
deconv_3=Deconv2D(4,kernel_size=(2,2),activation='relu')
upsamp_3=UpSampling2D((2,2))
deconv_4=Deconv2D(1,kernel_size=(2,2),activation='sigmoid')


model=conv_1(input_img)
model=conv_2(model)
model=conv_3(model)
model=deconv_1(model)
model=deconv_2(model)
model=deconv_3(model)
model=deconv_4(model)

model=Model(input_img,model)
model.summary()

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 4)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 8)     136       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 16)    528       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 32)    2080      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, None, None, 32)    4128      
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, None, None, 16)    2064      
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, None, None, 4)     260       
__________

In [14]:
training_x=np.array(x_train[:1]).reshape((1,256,256,4))
training_y=np.array(y_train[:1]).reshape((1,256,256,1))
validate_x=np.array(x_train[1]).reshape((1,256,256,4))
validate_y=np.array(y_train[1]).reshape((1,256,256,1))

from keras import backend as K
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
model.compile(optimizer = 'adam', 
                   loss = dice_coef_loss, 
                   metrics = [dice_coef, 'acc', 'mse'])

In [15]:
model.fit(x=training_x,y=training_y,epochs=3,validation_data=(validate_x,validate_y))

Train on 1 samples, validate on 1 samples
Epoch 1/3


InvalidArgumentError: Incompatible shapes: [65536] vs. [66049]
	 [[Node: metrics_2/dice_coef/mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](metrics_2/dice_coef/Reshape, metrics_2/dice_coef/Reshape_1)]]

Caused by op 'metrics_2/dice_coef/mul', defined at:
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/usr/local/lib/py